## Polarization along x, y and z axis
### Created by Kishwar-E Hasin

In [63]:
#!/usr/bin/python
import pandas as pd
import numpy as np
import math
from itertools import islice

# Put Filenames and paqrameters
high_symmetry_file_name = "HS"
low_symmetry_file_name = "CONTCAR"
output_file_name = "OUTCAR"
DISPLACEMENT_THRESHOLD = 0.89       # Change according to the positions of low symmetry POSCAR
PRINT_THRESHOLD = 0.000001          # Print if the absolute polarization is larger than this value
#=========================Change above file names and parameter

# Step 1: Load high symmetry and low symmetry POSCAR data and calculate displacement
with open(high_symmetry_file_name) as hs_lines:
    lattice_parameter = np.genfromtxt(islice(hs_lines, 2, 5))
    volume = np.prod(np.diag(lattice_parameter))
    for i, line in enumerate(hs_lines):
        if i == 1:
            n_atom = sum(map(int, line.strip().split()))
            break
    high_symmetry_position = np.genfromtxt(islice(hs_lines, 1, n_atom + 1))

with open(low_symmetry_file_name) as ls_file:
    low_symmetry_position = np.genfromtxt(islice(ls_file, 8, n_atom + 8))
    displacement_data = low_symmetry_position - high_symmetry_position
    #print (displacement_data)
    displacement_data = np.where(displacement_data > DISPLACEMENT_THRESHOLD, displacement_data - 1, displacement_data)
    displacement_data = np.where(displacement_data < -DISPLACEMENT_THRESHOLD, displacement_data + 1, displacement_data)
    #print (displacement_data)
    sum_ = np.sum(displacement_data, axis=0)
    if np.any(sum_ != 0):
        correction = sum_ / n_atom
        displacement_corrected = displacement_data - correction
    displacement = displacement_corrected * np.diag(lattice_parameter)

# Function to calculate polarization in a given direction
def calculate_polarization(direction_index, ion_index):
    displacement_to_calculate = displacement[:, direction_index]
    lookup_starts = [
        ' BORN EFFECTIVE CHARGES (in e, cummulative output)',  # For VASP 5
        ' BORN EFFECTIVE CHARGES (including local field effects) (in |e|, cummulative output)'  # For VASP 6
    ]
    lookup_end = ' INTERNAL STRAIN TENSOR FOR ION    1 for displacements in x,y,z  (eV/Angst):'

    capture_line = False
    outcar_data = []
    start_line_number = None

    with open(output_file_name) as outcar_file:
        for num, line in enumerate(outcar_file, 1):
            if any(lookup_start in line for lookup_start in lookup_starts):
                start_line_number = num
                capture_line = True
                continue
            if start_line_number and num == start_line_number + 1:
                continue
            if capture_line:
                if ' ion ' in line:
                    continue
                if f'    {ion_index}    ' in line:
                    outcar_data.append(line.strip())
            if lookup_end in line:
                capture_line = False
                break

    if not outcar_data:
        raise ValueError(f"No BORN effective charges found for ion index {ion_index}")

    born_effective_charge_to_calculate = [line.split()[ion_index] for line in outcar_data]
    total_dipole_moment = sum(float(displacement_to_calculate[i]) * float(born_effective_charge_to_calculate[i]) for i in range(len(born_effective_charge_to_calculate)))

    multiplication_factor = 1602.3  # Convert e/A^2 to uC/cm^2
    polarization = (multiplication_factor * total_dipole_moment) / volume
    return polarization


# Define labels for directions
labels = ['x', 'y', 'z']

# Calculate polarizations for x, y, and z directions and print them with labels
polarizations = []
for i in range(3):
    polarization = calculate_polarization(i, i + 1)
    if abs(polarization) > PRINT_THRESHOLD:
        print(f"Polarization, P{labels[i]}= {polarization} (uC/cm^2)")
    polarizations.append(polarization)
    
# Calculate and print total polarization
total_polarization = sum(polarizations)
print(f"Total Polarization, P = {total_polarization} (uC/cm^2)")


Polarization, Px= 6.549486095221103 (uC/cm^2)
Polarization, Py= -0.0006793724333228391 (uC/cm^2)
Polarization, Pz= 3.471460986603647e-05 (uC/cm^2)
Total Polarization, P = 6.548841437397646 (uC/cm^2)
